<a href="https://colab.research.google.com/github/MiqG/ColabSplice/blob/main/notebooks/Pangolin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pangolin**

## Model information
- [Github](https://github.com/tkzeng/Pangolin/tree/main)
- [Publication](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-022-02664-4)


## Tips and Instructions
- click the little ▶ play icon to the left of each cell below. Or Runtime > Run all to give it a try. This will produce a zip file called `test_SMN2.zip`, containing:
  - splice site strength predictions:
    - table: `test_SMN2-scores.tsv`
    - plot: `test_SMN2-splice_site_strength-line.pdf`
  - bigwig files that can be visualized through [IGV](https://igv.org/):
    - `test_SMN2-brain.bw`
    - `test_SMN2-heart.bw`
    - `test_SMN2-liver.bw`
    - `test_SMN2-testis.bw`
     
- The notebooks runs perfectly without GPU.

## Warnings
- Pangolin requires additional 5000 base pairs upstream and downstream to make predictions. For example, if the input sequence is 10003 bases long, Pangolin will only output a prediction for the 3 central positions.


In [ ]:
%%time
#@title ## Install packages (~1min 30s)
import os, time
if not os.path.isfile("finished_install"):
  # genomic file handling
  os.system("pip install -q pyfaidx==0.8.1.1 pyranges pyBigWig pyrle biopython")

  # install Pangolin
  os.system("pip install -q git+https://github.com/tkzeng/Pangolin.git")

  # install visualization
  os.system("pip install -q igv-notebook")

  os.system("touch finished_install")

# imports
import pyfaidx
import pyranges as pr
import hashlib, re, os
import torch
from tqdm import tqdm
import pandas as pd
import numpy as np
import pangolin
from pangolin.model import Pangolin, L, W, AR
import seaborn as sns
import matplotlib.pyplot as plt
import igv_notebook
import shutil
from google.colab import files
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

# notebook-wide variables
GENOME_FASTA_URLS = {
    "homo_sapiens (hg38-GencodeV44)": "https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_44/GRCh38.primary_assembly.genome.fa.gz"
}
# genome_gff_urls = {
#     "homo_sapiens (hg38-Ensembl113)": "https://ftp.ensembl.org/pub/release-113/gff3/homo_sapiens/Homo_sapiens.GRCh38.113.gff3.gz"
# }

# functions
def install_genome(species, genome_fasta_urls, fasta_file):
  if not os.path.isfile("finished_install_genome"):
    print(f"Downloading genome for {species}...")
    os.system(f"wget {genome_fasta_urls[species]} -O {fasta_file}")

    print("Uncompressing genome fasta...")
    os.system(f"gunzip {fasta_file}")

    os.system("touch finished_install_genome")

def get_hash(x):
  return hashlib.sha1(x.encode()).hexdigest()

def save_fasta(sequence, filename, header="sequence"):
    record = SeqRecord(Seq(sequence), id=header, description="")
    with open(filename, "w") as f:
        SeqIO.write(record, f, "fasta")

def reverse_complement(seq, padval_seq="N"):
    complement = str.maketrans(padval_seq+"ACGTacgt", padval_seq+"TGCAtgca")
    return seq.translate(complement)[::-1]

IN_MAP = np.asarray([[0, 0, 0, 0],
                     [1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])

def one_hot_encode(seq, strand):
    seq = seq.upper().replace('A', '1').replace('C', '2')
    seq = seq.replace('G', '3').replace('T', '4').replace('N', '0')
    if strand == '+':
        seq = np.asarray(list(map(int, list(seq))))
    elif strand == '-':
        seq = np.asarray(list(map(int, list(seq[::-1]))))
        seq = (5 - seq) % 5  # Reverse complement
    return IN_MAP[seq.astype('int8')]

def load_models():
  models = []
  for i in [0,2,4,6]:
    for j in range(1,4):
      model = Pangolin(L, W, AR)
      weights_file = os.path.join(pangolin.__path__[0],"models/final.%s.%s.3.v2" % (j, i))
      if torch.cuda.is_available():
        model.cuda()
        weights = torch.load(weights_file, weights_only=True)
      else:
        weights = torch.load(weights_file, map_location=torch.device('cpu'), weights_only=True)
      model.load_state_dict(weights)
      model.eval()
      models.append(model)

  return models

def predict(sequence, strand, models):

  print("Computing prediction across different tissues...")

  sequence = one_hot_encode(sequence, strand).T
  sequence = torch.from_numpy(sequence).float().unsqueeze(0)
  if torch.cuda.is_available():
    sequence = sequence.cuda()

  tissues = ["heart","liver","brain","testis"]
  tissue_scores = {}
  for j, tissue in tqdm(enumerate(tissues), total=len(tissues)):
    scores = []
    for model in models[3*j:3*j+3]:
      with torch.no_grad():
        score = model(sequence)[0][[1,4,7,10][j],:].cpu().numpy()
        if strand == '-':
          score = score[::-1]

        scores.append(score)

    tissue_scores[tissue] = np.mean(scores, axis=0)

  tissue_scores = pd.DataFrame(tissue_scores)

  return tissue_scores

In [ ]:
#@title ##run **Pangolin**
%%time

# user inputs
# @markdown Run settings
jobname = "test_SMN2" #@param {type:"string"}
jobname = re.sub(r'\W+', '', jobname)[:50]

predict_from = "genomic_coordinates" #@param ["custom_sequence","genomic_coordinates"]

download_outputs = True #@param {type:"boolean"}
igv_view = True #@param {type:"boolean"}

# @markdown ---

if predict_from == "genomic_coordinates":
  # @markdown Genomic coordinates (NOTE: requires installing genome at first run (~3min)):
  # coordinates
  chrom = "chr5" #@param {type:"string"}
  start = 70049669 #@param {type:"integer"}
  end = 70077595 #@param {type:"integer"}
  strand = "+" #@param ["-","+"]
  margin = 5000

  # genome
  species = "homo_sapiens (hg38-GencodeV44)" #@param ["homo_sapiens (hg38-GencodeV44)"]
  fasta_file = os.path.basename(GENOME_FASTA_URLS[species])
  install_genome(species, GENOME_FASTA_URLS, fasta_file)
  fasta_file = fasta_file.replace(".gz","")

  print("Loading genome data...")
  genome = pyfaidx.Fasta(fasta_file)
  # gtf = pr.read_gtf(gtf_file)
  sequence = genome[chrom][(start-1-margin):(end+margin)].seq

elif predict_from == "custom_sequence":
  # @markdown ---
  # @markdown Custom pre-mRNA sequence (NOTE: Pangolin will not make predictions on the first and last 5,000 bp):
  sequence = "" # @param {"type":"string","placeholder":"Paste nucleotide sequence here"}
  sequence = sequence.replace(" ","")
  chrom = f"{jobname}"
  strand = "+"
  margin = 5000

  # save custom sequence as fasta
  fasta_file = f"{jobname}.fa"
  save_fasta(sequence[margin:-margin], fasta_file, header=f"{jobname}")
  start = 1
  end = len(sequence) - margin*2

sequence = sequence.upper().replace("U","T")
assert set(sequence).issubset({"A", "C", "T", "G", "N"})

length = len(sequence)
print("length",length)

ID = jobname+"_"+get_hash(sequence)[:5]

# prep model(s)
if "models" not in dir():
  if "models" in dir():
    # delete old model from memory
    del models
    gc.collect()
    if torch.cuda.is_available():
      torch.cuda.empty_cache()

models = load_models()

# make prediction
print(f"Using {predict_from} to make prediction.")
scores = predict(sequence, strand, models)

# add more info
positions = np.arange(start, end+1)
if strand=="-":
  sequence = reverse_complement(sequence)[::-1]

scores["sequence"] = list(sequence[margin:-margin])
scores["position"] = positions
scores["chrom"] = chrom

# save
filename_scores = f"{jobname}-scores.tsv"
scores.to_csv(filename_scores, index=False, sep="\t")

In [ ]:
#@title PDF plot

X = scores.melt(id_vars=["chrom","position","sequence"], var_name="tissue")

g = sns.relplot(data=X, x="position", y="value", hue="tissue", row="tissue", kind="line", height=1.5, aspect=10)
g.set_xlabels("")
g.set_ylabels("", clear_inner=False)
g.figure.text(0.5, 0.02, "Nucleotide Position", ha="center", fontsize=10)
g.figure.text(0.0, 0.5, "Predicted Splice Site Strength", va="center", rotation="vertical", fontsize=10)

filename_plt = f"{jobname}-splice_site_strength-line.pdf"
g.figure.savefig(filename_plt, format="pdf", bbox_inches="tight")

plt.show()

In [ ]:
#@title Convert scores to BigWig (and IGV view)

# make bigwigs
tissues = ["heart", "liver", "brain", "testis"]
for tissue in tissues:
  ranges = scores[["chrom","position",tissue]].copy()
  ranges.columns = ["Chromosome","Start","Score"]
  ranges["Chromosome"] = ranges["Chromosome"]
  ranges["Start"] = ranges["Start"] - 1
  ranges["End"] = ranges["Start"] + 1
  ranges["Name"] = "U0"
  ranges["Score"] = ranges["Score"]#.round(2)
  ranges["Strand"] = strand
  ranges = ranges[["Chromosome","Start","End","Score","Name","Strand"]].copy()
  ranges = pr.PyRanges(ranges)
  chromosome_sizes = {chrom: len(genome[chrom]) for chrom in genome.keys()}
  ranges.to_bigwig(f"{jobname}-{tissue}.bw", chromosome_sizes, value_col="Score", rpm=False)
  # ranges.to_bigwig(dryrun=True, value_col="Score", rpm=False)

# make tracks
if igv_view:
  igv_notebook.init()

  if predict_from=="genomic_coordinates":
    b = igv_notebook.Browser({
        "reference": {
            "name": f"{species}",
            "fastaPath": f"./{fasta_file}",
            "indexPath": f"./{fasta_file}.fai"
        },
        "locus": f"{chrom}:{start}-{end}"
    })

    b.load_track({
          "name": "Gencode v44",
          "url": "https://igv-genepattern-org.s3.amazonaws.com/genomes/hg38/gencode.v44.basic.annotation.gff3.gz",
          "indexURL": "https://igv-genepattern-org.s3.amazonaws.com/genomes/hg38/gencode.v44.basic.annotation.gff3.gz.tbi",
          "order": 1,
          "format": "gff3",
          "type": "annotation"
    })

    b.load_track({
          "name": "Refseq Select",
          "format": "refgene",
          "url": "https://hgdownload.soe.ucsc.edu/goldenPath/hg38/database/ncbiRefSeqSelect.txt.gz",
          "indexed": False,
          "order": 2,
          "infoURL": "https://www.ncbi.nlm.nih.gov/gene/?term=$$",
          "type": "annotation",
          "height": 70
    })

  else:
    b = igv_notebook.Browser({
      "reference": {
          "name": "custom",
          "fastaPath": f"./{fasta_file}",
          "indexPath": f"./{fasta_file}.fai"
      },
      "locus": f"{chrom}:{start}-{end}"
    })

  colors = ["blue", "orange", "green", "red"]
  for tissue, color in zip(tissues, colors):
    b.load_track({
        "name": tissue,
        "path": f"./{jobname}-{tissue}.bw",
        "format": "bigwig",
        "type": "wig",
        "color": color,
        "graphType": "line",
    })

  b.zoom_in()

In [ ]:
#@title Download data

# prep outputs
## zip into a folder
output_folder = f"{jobname}-outputs"
os.makedirs(output_folder, exist_ok=True)

shutil.copy(filename_scores, os.path.join(output_folder, filename_scores))
shutil.copy(filename_plt, os.path.join(output_folder, filename_plt))
for tissue in tissues:
    shutil.copy(f"{jobname}-{tissue}.bw", os.path.join(output_folder, f"{jobname}-{tissue}.bw"))

shutil.make_archive(output_folder, 'zip', output_folder)

## download folder if user wants to
if download_outputs:
  files.download(f"{output_folder}.zip")